In [1]:
import fitz
import pandas as pd

In [2]:
with fitz.open("merged.pdf") as doc:
    text = []
    for page in doc:
        text = page.get_text().split("\n")
        print(page)
        print(text)

page 0 of merged.pdf
['X001IYJVB9', '4 Pack Hanging Railing ...alcony Porch Fence,Round', 'New', '']
page 1 of merged.pdf
['X001IYJVBJ', 'TIANTIAN 60 Shade Cloth ...Shade Fabric Accessories', 'New', '']
page 2 of merged.pdf
['X001IYJVBT', '4 Pack Round Hanging Ra...atio Balcony Porch Fence', 'New', '']
page 3 of merged.pdf
['X001IYJVC3', '2 Pack Hanging Railing P...lcony Porch Fence,Square', 'New', '']
page 4 of merged.pdf
['X001IYJVCD', '2 Pack Hanging Railing P...lcony Porch Fence,Square', 'New', '']
page 5 of merged.pdf
['X001IYJVCN', '6 Pcs Large Self-Watering...ng System for Plant,Heart', 'New', '']
page 6 of merged.pdf
['X001IYJZFB', 'Transparent Seedling Sta...row Starting Germination', 'New', '']
page 7 of merged.pdf
['X001IYJZFL', 'Hanging Railing Planter ... Stand with Folding Stand', 'New', '']
page 8 of merged.pdf
['X001IYJZFV', 'Transparent Seedling Sta...row Starting Germination', 'New', '']
page 9 of merged.pdf
['X001IYJZG5', '16 Pack 18" Plant Suppor...wist Ties for Flo

In [3]:
with fitz.open("merged.pdf") as doc:
    print(len(doc[28].get_text().split("\n")))

7


Page 28 looks problematic. For every other page, we've got a list of 4 elements. However, this one has 7.

In [4]:
# init empty dataframe with columns
df2=pd.DataFrame(columns=['fnsku','desc','status'])

# for loop to iterate through pdf pages
with fitz.open("merged.pdf") as doc:
    text = []
    for page in doc:
        text = page.get_text().split("\n")
        
        # if else logic test to catch the abnormal page 28, split the list and add a '' to the list with 3 elements.
        if len(text) == 7:
            text_1 = text[0:2]
            text_1.append("New")
            text_2 = text[3:6]
            df2 = df2.append(pd.DataFrame([text_1],columns=['fnsku','desc','status']),ignore_index = True)
            df2 = df2.append(pd.DataFrame([text_2],columns=['fnsku','desc','status']),ignore_index = True)
        else:
            df2 = df2.append(pd.DataFrame([text[0:3]],columns=['fnsku','desc','status']),ignore_index = True)

# removes duplicate rows
df2 = df2.drop_duplicates()
df2 = df2.dropna()

In [5]:
df2.head()

,fnsku,desc,status
0,X001IYJVB9,"4 Pack Hanging Railing ...alcony Porch Fence,R...",New
1,X001IYJVBJ,TIANTIAN 60 Shade Cloth ...Shade Fabric Access...,New
2,X001IYJVBT,4 Pack Round Hanging Ra...atio Balcony Porch F...,New
3,X001IYJVC3,"2 Pack Hanging Railing P...lcony Porch Fence,S...",New
4,X001IYJVCD,"2 Pack Hanging Railing P...lcony Porch Fence,S...",New


Looks good to me, now we need to concatenate desc and status together, with a row splitter between them.

In [6]:
df2["newdesc"] = df2["desc"] + "\r" +df2["status"]
df2.iloc[0,3]

'4 Pack Hanging Railing ...alcony Porch Fence,Round\rNew'

Exactly what we want, now we drop unwanted columns and export it to xlsx.

In [7]:
import xlsxwriter
writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')
df2.drop(columns=['desc', 'status']).to_excel(writer, sheet_name='Sheet1', index=False, header=False)
writer.save()